1. Introducción

El propósito de este documento es investigar casos de uso de la librería de Python PyCaret. Además se pondrá un ejemplo de estos y se dará una opinión al respecto del framework en cuestión.


2. PyCaret

PyCaret es una librería de Python que permite llevar a cabo desde la preparación de los datos, hasta el despliegue del modelo final en tan solo unos minutos. Esta librería es compatible con cualquier tipo de notebook de Python, y además nos permite realizar comparaciones de varios modelos automáticamente.

A modo de ejemplo vamos a crear un Jupyter Notebook que sea capaz, en tan solo unas líneas, de leer los datos, procesarlos obteniendo un ranking de modelos de ML, entrenar el modelo más potente y desplegarlo para obtener predicciones sobre datos.

Primero vamos a instalar PyCaret en nuestro entorno de Python, para ello ejecutamos el siguiente comando en una terminal:

Pero antes al leer la documentacion podemos ver que el pycaret solo funciona con python 3.6 ~ 3.8
asi que vamos a crear un entorno virtual con python 3.7

Instalamos el requirement.txt

In [ ]:
# %pip install pycaret[full]

In [ ]:
# !pip install pycaret --user

Hacemos los siguientes imports:

In [ ]:
import pandas as pd
import numpy as np
import pycaret
from pycaret.datasets import get_data
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Utilizaremos un conjunto de datos proporcionado por PyCaret llamado ‘credit’, para importarlo corremos el siguiente código

In [ ]:
index = get_data('index')

In [ ]:
data = get_data('credit')

In [ ]:
data.shape

Vamos a particionar el conjunto de datos, obteniendo el 95% para entrenar el modelo:

In [ ]:
df = data.sample(frac=0.95, random_state=42)
df

El 5% restante lo vamos a utilizar para comprobar el rendimiento del modelo sobre datos nunca antes vistos:

In [ ]:
df_unseen = data.drop(df.index)
df_unseen

Como podemos ver, tenemos varios predictores que se utilizarán para predecir la variable binaria ‘default’.

Lo último que haremos para limpiar los datos es resetear los índices de cada subconjunto de datos:

In [ ]:
df.reset_index(inplace=True, drop=True)
df_unseen.reset_index(inplace=True, drop=True)

Ahora vamos a comparar el rendimiento de distintos modelos. Para ello debemos, en primer lugar, importar

In [ ]:
from pycaret.classification import *

Paso seguido definimos el entorno de PyCaret con los datos de entrenamiento, esto hará que cada vez que llamemos a un modelo a entrenar se escojan dichos datos para entrenar. Además este proceso también preprocesa los datos automáticamente de manera que sea más fácil aplicar los modelos estadísticos:

In [ ]:
model_setup = setup(data=df, target='default', session_id=123, use_gpu=True)

Vemos como la ejecución de esta consulta es interactiva, pues espera que comprobemos que los tipos de datos inferidos automáticamente sean los correctos, en tal caso pulsamos enter. Entonces se nos mostrarán los cambios realizados a los datos de entrenamiento que hemos realizado

Podemos ver los modelos de clasificación de que dispone PyCaret, mediante el siguiente comando:

In [ ]:
models()

Este output es importante puesto que se necesitan los id de cada modelo para trabajar con ellos más en específico, como veremos a continuación.

Una de las funciones más útiles de esta librería nos permite comparar todos los modelos anteriores, esta función es la siguiente:

In [ ]:
best_model = compare_models()

Mediante esta tabla podemos escoger el modelo que más nos convenga, teniendo en cuenta las diferentes puntuaciones sobre las métricas que se muestran.

En nuestro caso, por ejemplo vamos a construir y entrenar un random forest sobre los datos de entrenamiento. Para ello ejecutamos:

In [ ]:
catboost = create_model('catboost')

In [ ]:
print(catboost)

Podemos ver como ha entrenado 10 modelos distintos, para poder obtener los detalles de este modelo en media y, así, poder extrapolar los resultados en mayor detalle. También se pueden ver los hiperparámetros con los que el modelo ha sido entrenado.

Para mejorar este modelo, es decir, obtener los hiperparámetros óptimos o que más se aproximan a estos, podemos correr la siguiente función, que entrena 10 modelos distintos 10 veces cada uno, y devuelve el que mejor precisión media obtenga:

In [ ]:
tunned_rf = tune_model(rf)

In [ ]:
print(tunned_rf)

In [ ]:
plot_model(tunned_rf,plot='auc')

In [ ]:
plot_model(tunned_rf, plot='feature')

Llegados a este punto, podemos obtener predicciones sobre el conjunto de datos test, que no ha sido utilizado para entrenar el modelo:

In [ ]:
unseen_prediction = predict_model(tunned_rf, data=df_unseen)
unseen_prediction.head()

Como podemos ver, se crean dos columnas nuevas. Label hace referencia a la predicción realizada mientras que Score es la probabilidad asociada a la predicción.

Por último, para terminar de configurar nuestro modelo random forest, debemos finalizar el modelo, es decir, se va a entrenar con todo el conjunto de datos del que se dispone:

In [ ]:
final_rf = finalize_model(tunned_rf)
print(final_rf)

De esta manera, el modelo está listo para su puesta en producción, por tanto podemos guardarlo localmente mediante:

In [ ]:
save_model(final_rf, 'modelo_final')

-------------------------------

In [ ]:
data = get_data('diamond')

In [ ]:
import plotly.express as px
fig = px.scatter(x=data['Carat Weight'], y= data['Price'], facet_col=data['Cut'], opacity= 0.25, template= 'plotly_dark', trendline_color_override= 'red', title='Diamonds')
fig.show();

In [ ]:
fig = px.histogram(data, x=["Price"], template = 'plotly_dark', title = 'Histogram of Price')
fig.show()

In [ ]:
data_copy = data.copy()
data_copy['Log_Price'] = np.log(data['Price'])
fig = px.histogram(data_copy, x=["Log_Price"], title = 'Histgrama Log Price', template = 'plotly_dark')
fig.show()

In [ ]:
from pycaret.regression import *
s = setup(data, target = 'Price', transform_target = True, log_experiment = True, experiment_name = 'diamond')

In [ ]:
best = compare_models()

In [ ]:
plot_model(best, plot = 'residuals_interactive')

In [ ]:
plot_model(best, plot = 'feature')


In [ ]:
final_best = finalize_model(best)
save_model(final_best, 'diamond-pipeline')

--------------------

In [2]:
from ydata_profiling import ProfileReport

/Users/angelg.villanueva/Documents/04_DATA/041_DATASCIENCE_BOOTCAMP/DS102024_2/3-MachineLearning/11-Extras/PyCaret/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = get_data('iris')

NameError: name 'get_data' is not defined

In [ ]:
profile = ProfileReport (data)

In [ ]:
s = setup(data, target='species', session_id= 123)

In [ ]:
eda()